In [118]:
!pip install PyDictionary

In [119]:
from PyDictionary import PyDictionary
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
# PlaintextParser, and LexRankSummarizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.summarizers.lex_rank import LexRankSummarizer

# initialize PyDictionary and stopwords list
dictionary = PyDictionary()
stop_words = set(stopwords.words("english"))


In [120]:
# function that returns the synonyms of a word
# read the jsonl file and convert it to a dictionary for easier access
import json
from tqdm import tqdm
with open('thesaurus.jsonl', 'r') as f:
    thesaurus = [json.loads(line) for line in f]

def get_synonyms(word):
    # iterate through the dictionary
    for entry in thesaurus:
        #print(entry)
        if entry['word'] == word:
            return entry['synonyms']
    # return None if the word is not in the dictionary
    return None

In [121]:
print(get_synonyms('world'))

['international', 'planetary', 'worldwide', 'global', 'world-wide']


In [122]:
def summarize_text(text, num_sentences):
    """
    Summarize the given text using the LSA or LexRank summarization algorithms and return the summary as a list of sentences
    """
    # create a PlaintextParser object to parse the text
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    # choose a summarization algorithm
    # algorithm = LsaSummarizer()
    algorithm = LexRankSummarizer()

    # summarize the text and return the summary as a list of sentences
    summary = algorithm(parser.document, num_sentences)

    return summary


In [127]:
from PyDictionary import PyDictionary
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from nltk.corpus import wordnet

# initialize PyDictionary and stopwords list
dictionary = PyDictionary()
stop_words = set(stopwords.words("english"))

def summarize_and_replace(paragraph, sentence, title):
    # summarize the paragraph
    place_holder = 'This '
    summary = summarize_text(paragraph, num_sentences=3)
    # convert the Sentence objects to strings and join them into a single string
    summary_text = " ".join([str(s) for s in summary])
    # create a set of the words in the title
    title_words = set(word_tokenize(title))
    # initialize a list to store the modified words
    modified_words = []
    # tokenize the summary text
    words = word_tokenize(summary_text)
    # iterate through the words
    for w in words:
        # check if the word is in the title
        if w in title_words and w not in stop_words:
            # find synonyms for the word
            #print(f'Debug: {w} needs to be replaced with a synonym')
            synonyms = get_synonyms(w.lower())
            #print(f'Debug: synonyms: {synonyms}')
            modified_word = w #!place_holder
        else:
            # use the original word if it is not in the title
            modified_word = w
        # append the modified word to the list of modified words
        modified_words.append(modified_word)
    # join the modified words into a string
    modified_text = " ".join(modified_words)
    # replace the original sentence with the modified text if it contains the placeholder 'This '
    modified_sentence = sentence.replace(sentence, modified_text) if place_holder in modified_text else sentence
    # find this pattern "This  , This  " and replace it with "This  "
    modified_sentence = re.sub(r'This\s+,\s+This', 'This', modified_sentence)
    # remove double spaces, double dots, and double commas
    modified_sentence = re.sub(r'\s+', ' ', modified_sentence) # remove double spaces
    modified_sentence = re.sub(r'\.\s+\.', '.', modified_sentence) # remove double dots
    modified_sentence = re.sub(r',\s+,', ',', modified_sentence) # remove double commas
    # remove any leading or trailing spaces that are followed by punctuation marks
    modified_sentence = re.sub(r'\s+([.,?!])', r'\1', modified_sentence)
    return modified_sentence


In [126]:
Example_Sentence = 'World War II'
Example_Paragraph = 'World War II, World War II or the Second World War, often abbreviated as WWII or WW2, was a world war that lasted from 1939 to 1945. It involved the vast majority of the worlds countriesincluding all of the great powersforming two opposing military alliances: the Allies and the Axis powers. World War II was a total war that directly involved more than 100 million personnel from more than 30 countries.\\nThe major participants in the war threw their entire economic, industrial, and scientific capabilities behind the war effort, blurring the distinction between civilian and military resources. Aircraft played a major role in the conflict, enabling the strategic bombing of population centres and deploying the only two nuclear weapons ever used in war.\\nWorld War II was by far the deadliest conflict in human history; it resulted in 70 to 85 million fatalities, mostly among civilians. Tens of millions died due to genocides including the Holocaust, starvation, massacres, and disease. In the wake of the Axis defeat, Germany and Japan were occupied, and war crimes tribunals were conducted against German and Japanese leaders.\\nThe causes of World War II are debated, but contributing factors included the Second Italo-Ethiopian War, Spanish Civil War, Second Sino-Japanese War, SovietJapanese border conflicts, rise of fascism in Europe and rising European tensions since World War I. World War II is generally considered to have begun on 1 September 1939, when Nazi Germany, under Adolf Hitler, invaded Poland. The United Kingdom and France subsequently declared war on Germany on 3 September. Under the MolotovRibbentrop Pact of August 1939, Germany and the Soviet Union had partitioned Poland and marked out their spheres of influence across Finland, Estonia, Latvia, Lithuania and Romania. From late 1939 to early 1941, in a series of campaigns and treaties, Germany conquered or controlled much of continental Europe, and formed the Axis alliance with Italy and Japan with other countries later. Following the onset of campaigns in North Africa and East Africa, and the fall of France in mid-1940, the war continued primarily between the European Axis powers and the British Empire, with war in the Balkans, the aerial Battle of Britain, the Blitz of the United Kingdom, and the Battle of the Atlantic. On 22 June 1941, Germany led the European Axis powers in an invasion of the Soviet Union, opening the Eastern Front, the largest land theatre of war in history.\\nJapan, which aimed to dominate Asia and the Pacific, was at war with the Republic of China by 1937. In December 1941, Japan attacked American and British territories with near-simultaneous offensives against Southeast Asia and the Central Pacific, including an attack on the US fleet at Pearl Harbor which resulted in the United States declaring war against Japan. The European Axis powers declared war on the United States in solidarity. Japan soon captured much of the western Pacific, but its advances were halted in 1942 after losing the critical Battle of Midway; later, Germany and Italy were defeated in North Africa and at Stalingrad in the Soviet Union. Key setbacks in 1943including a series of German defeats on the Eastern Front, the Allied invasions of Sicily and the Italian mainland, and Allied offensives in the Pacificcost the Axis powers their initiative and forced them into strategic retreat on all fronts. In 1944, the Western Allies invaded German-occupied France, while the Soviet Union regained its territorial losses and turned towards Germany and its allies. During 1944 and 1945, Japan suffered reversals in mainland Asia, while the Allies crippled the Japanese Navy and captured key western Pacific islands.\\nThe war in Europe concluded with the liberation of German-occupied territories and the invasion of Germany by the Western Allies and the Soviet Union, culminating in the Fall of Berlin to Soviet troops, Hitlers suicide, and the German unconditional surrender on 8 May 1945. Following the refusal of Japan to surrender on the terms of the Potsdam Declaration issued 26 July 1945, the United States dropped the first atomic bombs on the Japanese cities of Hiroshima on 6 August and Nagasaki on 9 August. Faced with an imminent invasion of the Japanese archipelago, the possibility of additional atomic bombings, and the Soviet Unions declared entry into the war against Japan on the eve of invading Manchuria, Japan announced on 10 August its intention to surrender, signing a surrender document on 2 September 1945.\\nWorld War II changed the political alignment and social structure of the globe. The United Nations was established to foster international co-operation and prevent future conflicts, with the victorious great powersChina, France, the Soviet Union, the United Kingdom, and the United Statesbecoming the permanent members of its Security Council. The Soviet Union and the United States emerged as rival superpowers, setting the stage for the nearly half-century-long Cold War. In the wake of European devastation, the influence of its great powers waned, triggering the decolonisation of Africa and Asia. Most countries whose industries had been damaged moved towards economic recovery and expansion. Political and economic integration, especially in Europe, began as an effort to forestall future hostilities, end pre-war enmities and forge a sense of common identity.'

# summarize the paragraph
summary = summarize_and_replace(Example_Paragraph, Example_Sentence, Example_Sentence)
# print the summary
print("Summary:")
print(summary)


Debug: World needs to be replaced with a synonym
Debug: synonyms: ['international', 'planetary', 'worldwide', 'global', 'world-wide']
Debug: War needs to be replaced with a synonym
Debug: synonyms: ['state of war']
Debug: II needs to be replaced with a synonym
Debug: synonyms: ['cardinal', '2', 'two']
Debug: World needs to be replaced with a synonym
Debug: synonyms: ['international', 'planetary', 'worldwide', 'global', 'world-wide']
Debug: War needs to be replaced with a synonym
Debug: synonyms: ['state of war']
Debug: II needs to be replaced with a synonym
Debug: synonyms: ['cardinal', '2', 'two']
Debug: World needs to be replaced with a synonym
Debug: synonyms: ['international', 'planetary', 'worldwide', 'global', 'world-wide']
Debug: War needs to be replaced with a synonym
Debug: synonyms: ['state of war']
Summary:
World War II
